In [2]:
import numpy as np
import pandas as pd
import datetime
import yfinance as yf
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# For importing universal scripts
import sys
import os
# Go up two levels from the subfolder
sys.path.append(os.path.abspath("../.."))
from indicators_returns import final_df #Universal script for indicator set and actuals

In [26]:
ticker = 'QQQ'
returns = [5, 10, 20, 30, 45, 60, 90]
lb = 20
df = final_df(ticker, returns, lb)
df = df.iloc[:-101].replace([np.inf, -np.inf], 0)

# Import ccsvs from earlier univariate analysis

In [59]:
cluster_df = pd.read_csv('../Indicator_Correlations/cluster.csv')  # includes 'indicator', 'cluster' columns
high_corr = pd.read_csv('../Indicator_Correlations/highly_correlated_pairs.csv')  # includes 'indicator_1', 'indicator_2'
low_corr = pd.read_csv('../Indicator_Correlations/low_corr_pairs.csv')  # includes 'indicator_1', 'indicator_2'
ind_dynamics_df = pd.read_csv('../Indicator_Dynamics/indicator_dynamics.csv')  # includes 'r', 'indicator', volatility/zscore/autocorr columns
logistic_results_df = pd.read_csv('../Univariate_Predictive_Power/logistic_results_xgb.csv')  # includes AUC, logloss, etc.

In [60]:
low_corr

,indicator_1,indicator_2,correlation
0,QQQ_SMA_10,QQQ_SMA_100,0.445600
1,QQQ_SMA_10,QQQ_EMA_100,0.498821
2,QQQ_SMA_10,QQQ_SMA_200,0.320216
3,QQQ_SMA_10,QQQ_EMA_200,0.365596
4,QQQ_SMA_10,Max_30_Rows_Since,-0.226220
...,...,...,...
16565,Intraday Change,Liquidity Score,-0.200630
16566,Intraday Volatility,Median Volume,0.491423
16567,Typical Price,Median Volume,-0.398034
16568,Median Volume,Median Close,-0.391725


# Filter out indicators with poor AUC or KS first

In [20]:
# Only keep if either AUC or KS is strong, and coef isn't completely flat
logi_filtered = logistic_results_df[
    (
        (logistic_results_df['flag'] != 'no_signal') |
        (logistic_results_df['ks_ts'] > .2)
    ) 
]

logi_filtered

,Unnamed: 0,r,class_balance,flag,indicator,auc_rs,auc_ts,log_loss_rs,log_loss_ts,coef_rs,coef_ts,ks_rs,ks_ts
0,0,5,N,keep,100_EMA_200,0.494549,0.424242,0.676573,0.675429,-0.517016,0.021185,0.051531,0.118791
1,1,10,N,keep,100_EMA_200,0.503271,0.395961,0.658263,0.653156,-0.072074,0.385507,0.049863,0.201726
2,2,20,N,keep,100_EMA_200,0.527863,0.654937,0.642633,0.616307,-0.009967,-0.081769,0.102268,0.294863
3,3,30,N,keep,100_EMA_200,0.544469,0.701299,0.626834,0.620562,-0.316896,-0.300490,0.131156,0.316488
4,4,45,N,keep,100_EMA_200,0.512505,0.669870,0.605028,0.596595,-0.648517,-0.240014,0.077935,0.370564
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420,3420,45,N,keep,vol_5_MA10,0.503338,0.556141,0.604589,0.594873,0.092188,0.062532,0.054201,0.157684
3424,3424,10,N,keep,vol_5_MA25,0.515533,0.550027,0.658223,0.652500,0.040007,0.004800,0.044251,0.132964
3427,3427,45,N,keep,vol_5_MA25,0.512318,0.556633,0.604223,0.593366,0.121215,0.092651,0.051115,0.162916
3441,3441,45,N,keep,vol_5_MA50,0.516008,0.555059,0.604317,0.593068,0.143012,0.111864,0.059223,0.174561


# Filter Based on Z-Score + Volatility

In [45]:
def is_too_flat(row):
    return all([
        row[f'tight_zrange_pct_{y}y'] > 0.1 for y in [5, 10, 15]
    ])

def is_too_noisy(row):
    return all([
        row[f'avg_vol_{w}'] > .8 for w in [21, 42, 63]
    ])

ind_dynamics_df['too_flat'] = ind_dynamics_df.apply(is_too_flat, axis=1)
ind_dynamics_df['too_noisy'] = ind_dynamics_df.apply(is_too_noisy, axis=1)

ind_dynamics_filtered = ind_dynamics_df[
    (~ind_dynamics_df['too_flat']) & (~ind_dynamics_df['too_noisy'])
].copy()

ind_dynamics_filtered

,r,indicator,avg_vol_21,avg_vol_42,avg_vol_63,tight_zrange_pct_5y,skew_5y,kurtosis_5y,tight_zrange_pct_10y,skew_10y,kurtosis_10y,tight_zrange_pct_15y,skew_15y,kurtosis_15y,autocorr_lag1,autocorr_lag5,autocorr_lag10,too_flat,too_noisy
0,5,QQQ_SMA_10,0.016324,0.018051,0.018748,0.0913,-1.060779,3.280899,0.1071,-1.074009,3.894510,0.0926,-1.156130,4.533029,0.810498,0.251337,0.012752,False,False
1,5,QQQ_EMA_10,0.013249,0.014828,0.015524,0.0706,-1.275087,3.887865,0.0754,-1.328596,4.671043,0.1093,-1.418590,5.514152,0.780154,0.332386,0.139805,False,False
2,5,QQQ_SMA_25,0.019650,0.024815,0.027178,0.0786,-1.247942,3.237450,0.0869,-1.302638,4.063127,0.0881,-1.339709,4.695366,0.925333,0.679746,0.415659,False,False
3,5,QQQ_EMA_25,0.016900,0.020614,0.022593,0.0714,-1.469925,4.045750,0.0786,-1.561139,5.278722,0.0749,-1.598159,5.687162,0.905915,0.648929,0.444341,False,False
4,5,QQQ_SMA_50,0.020345,0.027699,0.032400,0.0786,-1.371163,3.180743,0.0885,-1.479195,4.655209,0.1053,-1.439438,4.785597,0.962726,0.838188,0.693966,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3427,90,VIX_EA50,0.082370,0.100360,0.109736,0.0802,-0.853305,0.899264,0.0782,-0.611064,0.621182,0.0791,-0.669016,0.877496,0.902003,0.630895,0.436759,False,False
3436,90,Intraday Change,0.016979,0.017268,0.017461,0.0079,-0.334260,2.622368,0.0079,-0.771075,7.716115,0.0058,-0.687497,6.678580,-0.043567,-0.001382,0.006027,False,False
3437,90,Intraday Volatility,0.003257,0.004300,0.004896,0.1111,2.458413,8.068596,0.0893,2.895229,11.650239,0.0929,2.785761,11.315619,0.979837,0.855053,0.671851,False,False
3441,90,Liquidity Score,0.345174,0.386972,0.410124,0.1016,1.707394,5.679392,0.0964,1.878374,7.061227,0.1042,2.632067,11.859544,0.658589,0.370517,0.221622,False,False


# Filter out anything that fails the flat/noisy or low AUC test

## First block is an or across the two dfs

In [51]:
# Step 1: Build sets of (indicator, r) pairs from both filters
logi_pairs = set(zip(logi_filtered['indicator'], logi_filtered['r']))
dyn_pairs = set(zip(ind_dynamics_filtered['indicator'], ind_dynamics_filtered['r']))

# Step 2: Take union of both sets
valid_or_df = logi_pairs.union(dyn_pairs)
print(len(valid_or_df))

# Convert to DataFrame for easy counting
valid_or_df = pd.DataFrame(list(valid_or_df), columns=['indicator', 'r'])
print(valid_or_df.groupby('r')['indicator'].nunique())

2241
r
5     286
10    309
20    313
30    324
45    333
60    334
90    342
Name: indicator, dtype: int64


## Second block is an and across the two dfs

In [52]:
# These are indicators that PASSED each test
logi_pairs = set(zip(logi_filtered['indicator'], logi_filtered['r']))
dyn_pairs  = set(zip(ind_dynamics_filtered['indicator'], ind_dynamics_filtered['r']))

# Only keep indicators that passed BOTH filters (intersection)
valid_and_df = logi_pairs.intersection(dyn_pairs)
print(len(valid_and_df))

# Convert to DataFrame for easy counting
valid_and_df = pd.DataFrame(list(valid_and_df), columns=['indicator', 'r'])
print(valid_and_df.groupby('r')['indicator'].nunique())

704
r
5      69
10     72
20     74
30    112
45    141
60    105
90    131
Name: indicator, dtype: int64


# Greedy Forward Selection

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd
import numpy as np

def greedy_forward_selection(df, target_col, feature_list, cb='N', max_features=10, recent_test_size=500):
    selected_features = []
    remaining_features = feature_list.copy()
    history = []

    while len(selected_features) < max_features and remaining_features:
        best_score = -np.inf
        best_feature = None
        best_metrics = {}

        for feature in remaining_features:
            candidate_features = selected_features + [feature]

            # Prepare data
            X = df[candidate_features]
            y = df[target_col]
            valid = (~X.isna().any(axis=1)) & (~y.isna())
            X = X[valid]
            y = y[valid]

            if len(np.unique(y)) < 2 or len(y) < 600:
                continue

            # Temporal split
            X_test = X.iloc[:recent_test_size]
            y_test = y.iloc[:recent_test_size]
            X_train = X.iloc[recent_test_size:]
            y_train = y.iloc[recent_test_size:]

            if cb == 'Y':
                model = LogisticRegression(class_weight='balanced', max_iter=1000)
            else:
                model = LogisticRegression(max_iter=1000)

            model.fit(X_train, y_train)
            preds = model.predict(X_test)

            f1 = f1_score(y_test, preds)
            acc = accuracy_score(y_test, preds)

            # Class-specific metrics
            pos_mask = y_test == 1
            neg_mask = y_test == 0
            pos_acc = (preds[pos_mask] == y_test[pos_mask]).mean() if pos_mask.sum() > 0 else np.nan
            neg_acc = (preds[neg_mask] == y_test[neg_mask]).mean() if neg_mask.sum() > 0 else np.nan

            if f1 > best_score:
                best_score = f1
                best_feature = feature
                best_metrics = {
                    'f1': f1,
                    'accuracy': acc,
                    'pos_acc': pos_acc,
                    'neg_acc': neg_acc,
                    'pos_count': pos_mask.sum(),
                    'neg_count': neg_mask.sum()
                }

        if best_feature is None:
            break

        selected_features.append(best_feature)
        remaining_features.remove(best_feature)
        history.append((best_feature, best_metrics))

    return selected_features, pd.DataFrame([{
        'feature': feat,
        **metrics
    } for feat, metrics in history])


In [53]:
# Get features for this specific horizon
r = 10  # example
valid_features = valid_and_df[valid_and_df['r'] == r]['indicator'].tolist()

selected_feats, eval_df = greedy_forward_selection(df, f'Return_{r}', valid_features, cb='Y', max_features=10)

In [54]:
eval_df

,feature,f1,accuracy,pos_acc,neg_acc,pos_count,neg_count
0,10_EMA_200,0.709677,0.568,0.848875,0.105820,311,189
1,50_ESMA_100,0.737533,0.600,0.903537,0.100529,311,189
2,100_SMA_200,0.754765,0.614,0.954984,0.052910,311,189
3,50_SMA_100,0.770389,0.634,0.987138,0.052910,311,189
4,50_ESMA_200,0.773467,0.638,0.993569,0.052910,311,189
5,50_EMA_100,0.775000,0.640,0.996785,0.052910,311,189
6,vol_5_MA25,0.773467,0.638,0.993569,0.052910,311,189
7,100_ESMA_200,0.771930,0.636,0.990354,0.052910,311,189
8,vol_25_EA50,0.775000,0.640,0.996785,0.052910,311,189
9,25_ESMA_200,0.770389,0.634,0.987138,0.052910,311,189
